In [1]:
import pandas as pd

# Load all the sub-borough CSVs
income_df = pd.read_csv("sub-borougharea-incomediversityratio.csv")
racial_df = pd.read_csv("sub-borougharea-racialdiversityindex.csv")
education_df = pd.read_csv("sub-borougharea-populationaged25withabachelorsdegreeorhigher.csv")
carfree_df = pd.read_csv("sub-borougharea-car-freecommuteofcommuters.csv")
travel_df = pd.read_csv("sub-borougharea-meantraveltimetoworkminutes.csv")
poverty_df = pd.read_csv("sub-borougharea-povertyrate.csv")
nohs_df = pd.read_csv("sub-borougharea-populationaged25withoutahighschooldiploma.csv")
crowding_df = pd.read_csv("sub-borougharea-populationaged65.csv")
affordable_df = pd.read_csv("sub-borougharea-rentalunitsaffordableat80amiofrecentlyavailableunits.csv")
above65_df = pd.read_csv("sub-borougharea-populationaged65.csv")
under18_df = pd.read_csv("sub-borougharea-householdswithchildrenunder18yearsold.csv")
homeownership_df = pd.read_csv("sub-borougharea-homeownershiprate.csv")
born_df = pd.read_csv("sub-borougharea-borninnewyorkstate.csv")
new_df = pd.read_csv("communitydistrict-unitsauthorizedbynewresidentialbuildingpermits.csv")
foreclosure_df = pd.read_csv("sub-borougharea-noticesofforeclosureallresidentialproperties.csv")
median_income_df = pd.read_csv("sub-borougharea-medianhouseholdincome2023.csv")

# Helper function to keep and rename year columns
def select_and_prefix(df, prefix):
    year_cols = [col for col in df.columns if col.startswith("20")]
    renamed = {col: f"{prefix}_{col}" for col in year_cols}
    return df[["Sub-Borough Area"] + year_cols].rename(columns=renamed)

# Apply prefixing to avoid column collisions
income_df = select_and_prefix(income_df, "income")
racial_df = select_and_prefix(racial_df, "racial")
education_df = select_and_prefix(education_df, "education")
carfree_df = select_and_prefix(carfree_df, "carfree")
travel_df = select_and_prefix(travel_df, "travel")
poverty_df = select_and_prefix(poverty_df, "poverty")
nohs_df = select_and_prefix(nohs_df, "nohs")
crowding_df = select_and_prefix(crowding_df, "crowding")
affordable_df = select_and_prefix(affordable_df, "affordable")
above65_df = select_and_prefix(above65_df, "above65")
under18_df = select_and_prefix(under18_df, "under18")
homeownership_df = select_and_prefix(homeownership_df, "homeownership")
born_df = select_and_prefix(born_df, "born")
# Note: `new_df` may not have Sub-Borough Area; skipping unless mapped
foreclosure_df = select_and_prefix(foreclosure_df, "foreclosure")
median_income_df = select_and_prefix(median_income_df, "income_median")

# Merge all datasets on Sub-Borough Area
merged_df = income_df.merge(racial_df, on="Sub-Borough Area") \
                     .merge(education_df, on="Sub-Borough Area") \
                     .merge(carfree_df, on="Sub-Borough Area") \
                     .merge(travel_df, on="Sub-Borough Area") \
                     .merge(poverty_df, on="Sub-Borough Area") \
                     .merge(nohs_df, on="Sub-Borough Area") \
                     .merge(crowding_df, on="Sub-Borough Area") \
                     .merge(affordable_df, on="Sub-Borough Area") \
                     .merge(above65_df, on="Sub-Borough Area") \
                     .merge(under18_df, on="Sub-Borough Area") \
                     .merge(homeownership_df, on="Sub-Borough Area") \
                     .merge(born_df, on="Sub-Borough Area") \
                     .merge(foreclosure_df, on="Sub-Borough Area") \
                     .merge(median_income_df, on="Sub-Borough Area")

# Final merged DataFrame
df1 = merged_df

# Save to CSV (optional)
#df1.to_csv("all_years_gentrification_dataset.csv", index=False)

# Quick preview
print("Merged dataset shape:", df1.shape)
print(df1.head())

Merged dataset shape: (55, 282)
                       Sub-Borough Area  income_2005  income_2006  \
0                       Upper East Side     6.081081     5.514706   
1  Morningside Heights/Hamilton Heights     8.820000     8.255814   
2                        Central Harlem     7.900000     7.500000   
3                           East Harlem     5.931818     6.511765   
4             Washington Heights/Inwood     5.412490     5.451613   

   income_2007  income_2008  income_2009  income_2010  income_2011  \
0     5.395745     5.144033     5.909091     5.955335     5.938776   
1     7.806897     7.132353     6.962025     9.333333     7.158730   
2     6.181818     7.442308     7.870370     7.456140     5.600000   
3     6.327273     6.441441     6.818182     6.250000     6.800000   
4     4.827586     5.131579     6.302158     4.666667     4.993711   

   income_2012  income_2013  ...  income_median_2013  income_median_2014  \
0     5.000000     6.750000  ...       135013.664989    

In [23]:
features_df = df1[["Sub-Borough Area"] + [col for col in list(df1.columns)[1:] if (int(col[-4:]) >= 2005 and int(col[-4:]) <= 2019)]]

In [29]:
df = features_df.copy()
df["income_median_2005"] = pd.to_numeric(df["income_median_2005"], errors="coerce")
df["income_median_2019"] = pd.to_numeric(df["income_median_2019"], errors="coerce")
df['income_growth'] = (df['income_median_2019'] - df['income_median_2005']) / df['income_median_2005']
baseline_median_income = df['income_median_2005'].median()
median_growth = df['income_growth'].median()

def label_neighborhood(row, income_thresh, growth_thresh):
    if row['income_median_2005'] >= income_thresh:
        return "Higher-Income"
    else:
        if row['income_growth'] > growth_thresh:
            return "Gentrifying"
        else:
            return "Non-Gentrifying"

df['gentrification_label'] = df.apply(
    label_neighborhood,
    axis=1,
    args=(baseline_median_income, median_growth)
)

print(df[['Sub-Borough Area', 'income_median_2005', 'income_median_2019',
          'income_growth', 'gentrification_label']].head(15))


df.to_csv("gentrification_dataset_with_labels.csv", index=False)

                        Sub-Borough Area  income_median_2005  \
0                        Upper East Side              102540   
1   Morningside Heights/Hamilton Heights               34630   
2                         Central Harlem               33130   
3                            East Harlem               33350   
4              Washington Heights/Inwood               39340   
5                 Mott Haven/Hunts Point               17210   
6                     Morrisania/Belmont               23200   
7             Highbridge/South Concourse               25710   
8             University Heights/Fordham               24000   
9            Kingsbridge Heights/Mosholu               36180   
10                 Riverdale/Kingsbridge               59850   
11                 Soundview/Parkchester               41980   
12                Throgs Neck/Co-op City               54450   
13               Chelsea/Clinton/Midtown               87400   
14            Stuyvesant Town/Turtle Bay